## Notebook for crawling the data

[Common Crawl](https://commoncrawl.org/) is providing a [News Dataset](https://commoncrawl.org/2016/10/news-dataset-available/) with news from over 3 Million Feeds and News Sites. In total they possess nearly 9000 WARC files, each approximately 5 GB. 

This notebook extracts the most important information from these WARC files including: **URL**, **Text**, **Date**

*Important*: The notebook only takes into account the top 10 Newspapers in the USA.


#### Python Imports

In [ ]:
import requests
from newsplease import NewsPlease
import re
from warcio.archiveiterator import ArchiveIterator
import re
import os
import xlsxwriter
import pandas as pd
from bs4 import BeautifulSoup
import csv
from multiprocessing import Pool, Process

#### Index in which WARC_Files should be looking for

In [2]:
version = 8
start = 7822
end = 8000

#### All German news providers for which the corresponding articles in the WARC files are to be searched.

In [3]:
# Regex for finding any of the news provier articles in the WARC file
seeds_reg = ("^(http|https)://("
              #Links 
            "www.jungle.world|"
            "www.taz.de|"
            "www.vice.com|"
            "www.rubikon.news|" 
            "www.de.indymedia.org|" 
            "www.neues-deutschland.de|" 
            "www.nrhz.de|" 
            "www.arte.tv|"
            "www.pressenza.com|"
            "www.republik.ch|" 
            "www.zeitpunkt.ch|"
            "www.jungundnaiv.de|" 
            "www.infosperber.ch|" 
            "www.amerika21.de|" 
            "www.jungewelt.de|" 
            "www.woz.ch|"
            "www.wsws.org|" 
            "www.freitag.de|" 
            "www.nachdenkseiten.de|"
             
            #Halb Rechts
            "www.welt.de|"
            "www.blick.ch|" 
            "www.bild.de|"
            "www.krone.at|"
            "www.nuoviso.tv|"
            "www.de.sputniknews.com" 
            "www.expresszeitung.com|"
            "www.epochtimes.de|"
            "www.deutsche-wirtschafts-nachrichten.de|" 
            "www.diepresse.com|" 
            "www.20min.ch|" 
            "www.focus.de|" 
            "www.faz.net|" 
             
            #Central 
            "www.n-tv.de|"
            "www.handelsblatt.com|"
            "www.derstandard.at|"
            "www.neopresse.com|"
            "www.uncut-news.ch|"
            "www.wiwo.de|"
            "www.zeit-fragen.ch|"
            "www.blaetter.de|"
            "www.antikrieg.com|"
            "www.zeit.de|" 
            "www.cashkurs.com|"
            "www.anti-spiegel.ru|"
            "www.multipolar-magazin.de|"
            "www.hintergrund.de|" 
            "www.deutschlandradio.de|"
            "www.ard.de|"
            "www.zdf.de|"
            "www.sueddeutsche.de|" 
            "www.deutschlandfunk.de|"
            "www.spiegel.de|"
            "www.tagesspiegel.de|" 
             
            #Halb Links
            "www.srf.ch|"
            "www.orf.at|"
            "www.neue-debatte.com|" 
            "www.krautreporter.de|"
            "www.voltairenet.org|" 
            "www.kenfm.de|" 
            "www.heise.de|" 
            "www.free21.org|"  
            "www.journal21.ch|"

            #Rechts
            "www.weltwoche.ch|"
            "www.achgut.com|"
            "www.aargauerzeitung.ch|" 
            "www.parstoday.com|"
            "www.info-direkt.eu|"
            "www.china.org.cn|"
            "www.zuerst.de|"
            "www.geolitico.de|"
            "www.ef-magazin.de|" 
            "www.contra-magazin.com|" 
            "www.sezession.de|"
            "www.freiewelt.net|"
            "www.national-zeitung.de|"
            "www.unzensuriert.at|"  
            "www.compact-online.de|"
            "www.tichyseinblick.de|"
            "www.jungefreiheit.de|"
            "www.mmnews.de|"
            "www.cicero.de)")

# Regex to find the article's url
name_reg = "([^\/]+$)"

#### List in which all news providers are classified to one of the five political positions.

In [6]:
rechts = ["www.weltwoche.ch", "www.achgut.com", "www.aargauerzeitung.ch", "www.parstoday.com",
          "www.info-direkt.eu", "www.zuerst.de", "www.china.org.cn", "www.geolitico.de", "www.ef-magazin.de", 
          "www.contra-magazin.com", "www.sezession.de", "www.freiewelt.net", "www.national-zeitung.de", 
          "www.unzensuriert.at", "www.compact-online.de", "www.tichyseinblick.de", "www.jungefreiheit.de",
          "www.mmnews.de", "www.cicero.de"]
print(len(rechts))


halbRechts = ["www.welt.de", "www.blick.ch", "www.bild.de", "www.krone.at", 
              "www.de.sputniknews.com", "www.expresszeitung.com", "www.nuoviso.tv", "www.deutsche-wirtschafts-nachrichten.de",
              "www.epochtimes.de", "www.diepresse.com", "www.20min.ch", "www.focus.de", "www.faz.net"]
print(len(halbRechts))


central = ["www.n-tv.de", "www.handelsblatt.com", "www.antikrieg.com", "www.derstandard.at", "www.neopresse.com",
           "www.uncut-news.ch", "www.zeit-fragen.ch","www.zeit.de", "www.wiwo.de", "www.spiegel.de", "www.blaetter.de", "www.anti-spiegel.ru", "www.cashkurs.com", 
           "www.multipolar-magazin.de", "www.deutschlandradio.de", "www.hintergrund.de", "www.zdf.de", "www.ard.de",
            "www.sueddeutsche.de", "www.deutschlandfunk.de", "www.tagesspiegel.de"]
print(len(central))


halbLinks = ["www.srf.ch", "www.orf.at", "www.neue-debatte.com", "www.krautreporter.de", 
             "www.kenfm.de", "www.heise.de", "www.free21.org", "www.voltairenet.org", "www.journal21.ch"]
print(len(halbLinks))


links = ["www.jungle.world", "www.taz.de", "www.vice.com", "www.pressenza.com", "www.rubikon.news", "www.de.indymedia.org", 
         "www.neues-deutschland.de", "www.nrhz.de", "www.arte.tv",
         "www.republik.ch", "www.zeitpunkt.ch", "www.jungundnaiv.de", 
         "www.infosperber.ch", "www.amerika21.de", "www.jungewelt.de", "www.woz.ch",
         "www.wsws.org", "www.freitag.de", "www.nachdenkseiten.de"]
print(len(links))

19
13
21
9
19


#### Reading all downloaded WARC-Files URLs from commoncrawl.org

In [ ]:
urls = open('Urls/Urls_All_Sept.txt', "r")
url_lines = urls.readlines()
urls.close()

In [ ]:
print(len(url_lines))

79600


#### Generates a new TXT file containing all URLs to the WARC files to be searched.

In [ ]:
with open('Urls/Urls_Sept_Part_'+str(version)+'.txt', 'w', newline='') as f:
    for i in range(start,end+1):
        f.write(url_lines[i])
        
print(start)
print(end)
#start = end
#end = end + len(url_lines)//24

7822
8000


#### Function that searches only for German articles and reads the text

In [ ]:
def get_MainTextfromURL(uri):
    try:
        article = NewsPlease.from_url(uri)
        if not article.maintext:
            article = "NONE"
        elif (" der " in article.maintext.lower() or " die " in article.maintext.lower() or
            " das " in article.maintext.lower() or " sie " in article.maintext.lower() or
            " und " in article.maintext.lower() or " von " in article.maintext.lower()):
                article = article.maintext
        else:
            article = "NONE"
    except:
        article = "NONE"
    return article

### Processing

In [ ]:
filepath = 'GeneratedFiles/Data_WARC_September_Part_'+str(version)
#try:
#    os.remove(filepath+".csv")
#except:
#    pass

#### Checks if the currently found message is one of our searched ones and makes the appropriate classification.

In [ ]:
def extract_information(url):
    """
    Extracts information from the given url and stores them in a .txt file named after the url
    
    :param: url: Url of the WARC file
    """
    resp = requests.get(url, stream=True)
    filename = re.findall(name_reg, url)[0][0:-8]
    
    with open(filepath+'.csv'.format(filename),'a', newline='') as file:
        writer = csv.writer(file)
        #writer.writerow(["publisher", "uri", "article", "label"])
        index = 0
        label = ""
        
        for record in ArchiveIterator(resp.raw):
            if record.rec_type == 'response':
                # Returns the URI
                uri = str(record.rec_headers.get_header('WARC-Target-URI'))

                if (re.search(seeds_reg, uri) is not None):
                    date = record.rec_headers.get_header('WARC-Date')
                    print("Date {}".format(date))
                    publisher = re.search("(www.+?(?=\/))", uri)
                    article = get_MainTextfromURL(uri)
                    print("Publisher: "+publisher.group())
                    print("URI {}".format(uri))
                    for journal in rechts:
                        if journal in uri:
                            label = "rechts"
                            
                    if not label:
                        for journal in halbRechts:
                            if journal in uri:
                                label = "halbRechts"
                                
                    if not label:
                        for journal in central:
                            if journal in uri:
                                label = "central"
                                
                    if not label:
                        for journal in halbLinks:
                            if journal in uri:
                                label = "halbLinks"
                                
                    if not label:
                        for journal in links:
                            if journal in uri:
                                label = "links"
                                
                    print("Label: "+label+"\n")                
                   

                    writer.writerow([publisher.group(), uri, article, label])
                    
                    label = ""
                    index = index + 1
                    
                    # Just for FoxNews
                    #contents_p = soup.find_all("p", {"class": "speakable"})
                    #for content_p in contents_p:
                        #print(content_p[0].text)

#### Runs through all messages found in the WARC files.

In [ ]:
%%time
# Regex to find the url name
url_reg = "(?<=crawl-data)(.*)"
url_root = "https://commoncrawl.s3.amazonaws.com/crawl-data"

# For each url, stream the WARC file and extract the information we need
urls = open('Urls/Urls_Sept_Part_'+str(version)+'.txt', "r")
url = urls.readline()
while url:
    filename = re.findall(url_reg, url)[0]
    print("Url: {} \n".format(url_root + filename))
    try:
        extract_information(url_root + filename)
        url = urls.readline()
    except:    
        url = urls.readline()
urls.close()

Url: https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/segments/1600400192778.51/warc/CC-MAIN-20200919142021-20200919172021-00222.warc.gz 

Date 2020-09-19T15:03:41Z
Publisher: www.aargauerzeitung.ch
URI https://www.aargauerzeitung.ch/sport/fussball/gewisse-zahlen-rund-um-gc-stimmen-nachdenklich-134172102
Label: rechts

Date 2020-09-19T16:39:52Z
Publisher: www.deutschlandfunk.de
URI https://www.deutschlandfunk.de/dlf-audio-archiv.2386.de.html?cal:month=11&cal:year=2019&drau:broadcast_id=171&drau:from=24.11.2019&drau:to=24.11.2019
Label: central

Date 2020-09-19T16:11:01Z
Publisher: www.deutschlandfunk.de
URI https://www.deutschlandfunk.de/dlf-audio-archiv.2386.de.html?cal:month=4&cal:year=2019&drau:from=28.04.2019&drau:page=3&drau:to=28.04.2019
Label: central

Date 2020-09-19T16:00:48Z
Publisher: www.diepresse.com
URI https://www.diepresse.com/5848693/apple-marketingchef-phil-schiller-tritt-zuruck?from=rss
Label: halbRechts

Date 2020-09-19T16:59:04Z
Publisher: www.tagess

not a 200 response: 404


Publisher: www.arte.tv
URI https://www.arte.tv/de/videos/096847-000-A/israel-arabische-kunstrebellinen/
Label: links

Date 2020-09-19T16:32:47Z
Publisher: www.cicero.de
URI https://www.cicero.de/comment/31937
Label: rechts

Date 2020-09-19T16:35:24Z
Publisher: www.deutschlandfunk.de
URI https://www.deutschlandfunk.de/programmvorschau.281.de.html?cal:month=8&cal:year=2019&drbm:date=22.08.2019
Label: central

Date 2020-09-19T16:51:31Z
Publisher: www.freitag.de
URI https://www.freitag.de/autoren/der-freitag/auf-nummer-sicher-1
Label: links

Date 2020-09-19T15:05:36Z
Publisher: www.freitag.de
URI https://www.freitag.de/autoren/windheuser/netzschau-die-ausweitung-der-gefangniszone
Label: links

Date 2020-09-19T16:29:52Z
Publisher: www.srf.ch
URI https://www.srf.ch/news/regional/basel-baselland/projekt-filmemacher-residenz-basler-wohnung-fuer-filmschaffende-aus-aller-welt
Label: halbLinks

Date 2020-09-19T17:13:29Z
Publisher: www.sueddeutsche.de
URI https://www.sueddeutsche.de/bayern/schwand

not a 200 response: 503


Date 2020-09-19T16:59:10Z
Publisher: www.tagesspiegel.de
URI https://www.tagesspiegel.de/politik/mirko-norac-protestmaersche-der-trotz-der-vaterlaendischen-krieger/203282.html
Label: central

Date 2020-09-19T15:12:37Z


...

Publisher: www.tagesspiegel.de
URI https://www.tagesspiegel.de/politik/berlin-will-220-kosovo-fluechtlinge-aufnehmen/74862.html
Label: central

Date 2020-09-19T16:47:25Z
Publisher: www.tagesspiegel.de
URI https://www.tagesspiegel.de/wirtschaft/kommt-die-reform-der-reform-streit-um-scheinselbstaendigkeit/76128.html
Label: central

Date 2020-09-19T15:17:40Z
Publisher: www.welt.de
URI https://www.welt.de/print/wams/article120029021/Syrien-Russland-und-USA-einig.html
Label: halbRechts

Date 2020-09-19T15:16:20Z
Publisher: www.wsws.org
URI https://www.wsws.org/en/articles/2009/04/gree-a20.html
Label: links

Url: https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/segments/1600400192778.51/warc/CC-MAIN-20200919142021-20200919172021-00372.warc.gz 

Date 2020-09-19T15:32:34Z
Publisher: www.bild.de
URI https://www.bild.de/bild-plus/politik/inland/politik-inland/abschiebung-aus-tuerkei-wie-gefaehrlich-ist-erdogans-terror-familie-66035592,view=conversionToLogin.bild.html
Label: halbRe

not a 200 response: 502


Publisher: www.diepresse.com
URI https://www.diepresse.com/5867581/erst-messi-dann-die-welt-und-als-zugabe-noch-der-mond
Label: halbRechts

Date 2020-09-19T16:47:28Z
Publisher: www.focus.de
URI https://www.focus.de/familie/eltern/orkan-in-deutschland-hier-bleiben-schulen-am-dienstag-geschlossen_id_11641343.html
Label: halbRechts

Date 2020-09-19T16:49:35Z
Publisher: www.krone.at
URI https://www.krone.at/2180537
Label: halbRechts

Date 2020-09-19T16:08:25Z
Publisher: www.n-tv.de
URI https://www.n-tv.de/politik/Aussteiger-Bosbach-wird-zum-Hit-article19935676.html
Label: central

Date 2020-09-19T16:53:21Z
Publisher: www.n-tv.de
URI https://www.n-tv.de/politik/Russland-verlegt-Kampfjets-auf-die-Krim-article20778032.html
Label: central

Date 2020-09-19T16:34:48Z
Publisher: www.neues-deutschland.de
URI https://www.neues-deutschland.de/artikel/159086.die-heuschrecke-schwirrt-ab.html
Label: links

Date 2020-09-19T16:16:25Z
Publisher: www.srf.ch
URI https://www.srf.ch/sport/motorsport/formel-1/

not a 200 response: 404


Publisher: www.welt-der-frauen.at
URI https://www.welt-der-frauen.at/wie-die-zeit-vergeht/
Label: 

Url: https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/segments/1600400192778.51/warc/CC-MAIN-20200919142021-20200919172021-00400.warc.gz 

Date 2020-09-19T16:39:35Z
Publisher: www.deutschlandradio.de
URI https://www.deutschlandradio.de/audio-archiv.260.de.html?cal:month=10&cal:year=2018&drau:from=30.11.2018&drau:page=4&drau:to=30.11.2018
Label: central

Date 2020-09-19T16:38:57Z
Publisher: www.deutschlandradio.de
URI https://www.deutschlandradio.de/audio-archiv.260.de.html?cal:month=12&cal:year=2018&drau:broadcast_id=&drau:from=15.01.2019&drau:searchterm=reintjes&drau:station_id=0&drau:submit=1&drau:to=15.01.2019
Label: central

Date 2020-09-19T16:30:11Z
Publisher: www.deutschlandradio.de
URI https://www.deutschlandradio.de/audio-archiv.260.de.html?cal:month=9&cal:year=2014&drau:from=06.10.2014&drau:to=06.10.2014
Label: central

Date 2020-09-19T16:19:30Z
Publisher: www.hei

### Cleansing

In [ ]:
df = pd.read_csv(filepath+'.csv'.format(filename), sep=',',header=None)
df.columns = ['publisher', 'uri', 'article', 'label']
df = df[['publisher', 'uri', 'article', 'label']]
df = df[df.publisher != "www.pressenza.com"]
df = df[df.article != "NONE"]
df = df[~df.label.isnull()]

### Write Excel

In [ ]:
dfLinks = df[df.label == "links"]
dfHalbLinks = df[df.label == "halbLinks"]
dfCentral = df[df.label == "central"]
dfHalbLRechts = df[df.label == "halbRechts"]
dfRechts = df[df.label == "rechts"]

# Create a Pandas Excel writer using XlsxWriter as the engine.
try:
    os.remove(filepath+'.xlsx')
except:
    pass
writer = pd.ExcelWriter(filepath+'.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, index = False, sheet_name='AlleDatensätze')
dfLinks.to_excel(writer, index = False, sheet_name='Links')
dfHalbLinks.to_excel(writer, index = False, sheet_name='Halb links')
dfCentral.to_excel(writer, index = False, sheet_name='Central')
dfHalbLRechts.to_excel(writer, index = False, sheet_name='Halb rechts')
dfRechts.to_excel(writer, index = False, sheet_name='Rechts')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

### Write CSV

Writing again an CSV-File with correct Headers

In [ ]:
try:
    os.remove(filepath+"_GeneratedFromExcel.csv")
except:
    pass 
# Write the dataframe object into csv file
df.to_csv(filepath+"_GeneratedFromExcel.csv", index=None, header=True)